In [ ]:
import pandas as pd
from tqdm import  tqdm

<h1>Préparation des données partie 2</h1>
Dans cette partie nous allons faire la construction du vocabulaire et extraction des caractéristiques

In [ ]:
data=pd.read_csv('./data.csv')
X=data['text']
X.head()

<h2>1-Construction du Vocabulaire<h2>
<h3>1.1 Géneration du vocabulaire<h3>


In [ ]:
def update_word_list(email:str,word_list:dict):
    #Extraire tous les mots de l'email et les mettre dans le dictionnaire vocabulaire.
    #Si le mot existe incrementer le nombre d'apparitions sinon insert le nouveau mot dans le dictionnaire et initialiser la valeur d'apparition a 1

In [ ]:
word_list=dict() #dictionnaire du vocabulaire sous la forme {mot : nombre d'apparition}
for i in tqdm(range(len(X))):
    update_word_list(X[i],word_list)


In [ ]:
def build_vocabulary(word_list:dict,k:int)->list:
    vocabulary=list()
    #Construire le vocabulaire en prenant les mots dont le nombre d'apparitions est supérieure à 'k'
    #Note : Faire plusieurs tests sur k pour avoir un dictionnaire entre 10000 et 50000
    return  vocabulary

In [ ]:
#Génération du vocabulaire dans un fichier
k=1
vocabulary=build_vocabulary(word_list, k)
with open('vocab.txt','w') as f:
    for i in vocabulary:
        f.write(f"{i}\n")


<h3>1.2 Indéxation des mots de chaque mail</h3>

In [7]:
def get_vocabulary(filename:str)->list:
    with open(filename, 'r') as file:
        # lire le fichier en tant que texte
        text = file.read()
        # convertir le texte en liste de mots
        vocabulary_list = text.split()
    return vocabulary_list

In [8]:
def indexation(vocabList:list,email:str)->list:
     index=list()
    #Découper l'email en tableau de mot
     words = email.split()
    #Parcourir chaque mot dans l'email et trouver son index dans la liste de vocabulaire
     for word in words:
        if word in vocabList:
            #Ajouter l'index du mot dans la liste d'index de mots
            index.append(vocabList.index(word))
     return index

In [9]:
#just a random test
# test_str = "sing hello to the world eat pizza and chocolate enjoy the sunshine read a book and watch a movie"
# vocabList = get_vocabulary('test.txt')
# indexation(vocabList,test_str)

[9, 1, 8, 4, 13, 19]

In [ ]:
def generateAllIndexes(vocabList:list,emails)->list:
    list_index=list()
    for i in tqdm(range(len(emails))):
        list_index.append(indexation(vocabulary,X[i]))
    return list_index

<h2>Extraction des caractéristiques</h2>
Représentation des caractéristiques par comptage

In [ ]:
def extract_car(vocabList:list,index:list)->list:
    # Créer une liste vide de taille meme taille que vocabList
    caracteristic_vect = [0] * len(vocabList)
    #Utiliser l'index créé dans l'étape d'indexation pour remplir le vecteur des caractéristique la taille du vecteur est eagle au nombre de mots du 
    # vocabulaire et qui sera initialiser à 0 et à chaque apparition du i eme mot faire la mise a jour du vecteur caracteristiques
    #Parcourir chaque element dans index 
    for i in range(len(index)):
            #incrementer la case[index] de 1 dans caracteristic_vect
            caracteristic_vect[index[i]]+=1
            
    return caracteristic_vect

In [ ]:
#just a random test
test_str = "hello  the truth will set you free , free  "
vocabList = get_vocabulary('test.txt')
index=indexation(vocabList,test_str)
print(index)
extract_car(vocabList,index)

In [ ]:
def extract_car_emails(vocabList:list,emails:list)->list:
    allEmailsCarac=list()
    for i in tqdm(range(len(X=emails))):
        allEmailsCarac.append(extract_car(vocabList,emails[i]))
    return allEmailsCarac